# Utils

In [ ]:
import os
import numpy as np
import pandas as pd
import nibabel as nib
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, mean_squared_error, r2_score
from sklearn.metrics import precision_recall_curve, roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import StratifiedKFold

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Paths
path_labels = '/kaggle/input/fcmaps-metadata/labels.csv'
path_fcmaps = '/kaggle/input/fcmaps-processed'                   
path_fcmaps_augmented = '/kaggle/input/fcmaps-augmented-processed/FCmaps_augmented_processed'

# Load labels
df_labels = pd.read_csv(path_labels)

# Temporary:
to_exclude = ['3_S_5003', '4_S_5003', '4_S_5005', '4_S_5007', '4_S_5008']
df_labels = df_labels[~df_labels['ID'].isin(to_exclude)].reset_index(drop=True)

In [ ]:
# Parameters
batch_size = 16
epochs = 20
lr = 1e-3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
task = 'classification'               # or 'regression'
n_folds = 10

if task == 'classification':
    criterion = nn.CrossEntropyLoss()
    n_classes = 2  
    label_column = 'Group'
else:
    criterion = nn.MSELoss()
    n_classes = 1
    #label_column = 'Age'

# Dataset

In [ ]:
class FCDataset(Dataset):
    def __init__(self, data_dir, df_labels, label_column, task, transform=None):       
        self.data_dir = data_dir
        self.df_labels = df_labels.reset_index(drop=True)
        self.label_column = label_column
        self.task = task
        self.transform = transform
    
        # Dictionary for mapping strings to indices if labels are not numbers
        if not pd.api.types.is_numeric_dtype(self.df_labels[self.label_column]):
            unique_labels = sorted(self.df_labels[self.label_column].unique())
            self.label_mapping = {label: i for i, label in enumerate(unique_labels)}
        else:
            self.label_mapping = None
            
        self.samples = []

        # Loop over each row of the dataframe
        for _, row in self.df_labels.iterrows():
            subj_id = row['ID']
            
            if self.task == 'classification':
                # Classification: map the labels
                label = self.label_mapping[row[self.label_column]]
            
            else:  
                # Regression: convert to float
                label = float(row[self.label_column])

            # Reconstruct the file paths 
            file_path = os.path.join(data_dir, f"{subj_id}.processed.npy")
            if os.path.exists(file_path):
                self.samples.append((file_path, label))
            else:
                print(f"Missing file: {file_path}")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        file_path, label = self.samples[idx]
        
        # Load and reshape the volume: (1, 91, 109, 91)
        volume = np.load(file_path)
        volume = np.expand_dims(volume, axis=0)  

        # Covert volume into a tensor
        x = torch.tensor(volume, dtype=torch.float32)

        # Convert the label into a tensor
        if self.task == 'classification':
            y = torch.tensor(label, dtype=torch.long)
        else: 
            y = torch.tensor(label, dtype=torch.float32)

        if self.transform:
            x = self.transform(x)

        return x, y

In [ ]:
class AugmentedFCDataset(Dataset):
    def __init__(self, data_dir, df_labels, label_column, task, transform=None):
        
        self.data_dir = data_dir
        self.df_labels = df_labels.reset_index(drop=True)
        self.label_column = label_column
        self.task = task
        self.transform = transform

        # Mapping
        if not pd.api.types.is_numeric_dtype(self.df_labels[self.label_column]):
            unique_labels = sorted(self.df_labels[self.label_column].unique())
            self.label_mapping = {label: i for i, label in enumerate(unique_labels)}
        else:
            self.label_mapping = None
            
        self.samples = []

        for _, row in self.df_labels.iterrows():
            subj_id = row['ID']

            if self.task == 'classification':
                label = self.label_mapping[row[self.label_column]]
            else:
                label = float(row[self.label_column])

            # Loop over each subject folder and list each augmentation
            subject_folder = os.path.join(data_dir, subj_id)
            if os.path.isdir(subject_folder):
                for file in os.listdir(subject_folder):
                    if file.endswith('.npy'):
                        file_path = os.path.join(subject_folder, file)
                        self.samples.append((file_path, label))
            else:
                print(f"Warning: missing augmented folder for subject {subj_id}")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        file_path, label = self.samples[idx]

        # Load and reshape the volume: (1, 91, 109, 91)
        volume = np.load(file_path)
        volume = np.expand_dims(volume, axis=0)

        x = torch.tensor(volume, dtype=torch.float32)

        if self.task == 'classification':
            y = torch.tensor(label, dtype=torch.long)
        else:
            y = torch.tensor(label, dtype=torch.float32)

        if self.transform:
            x = self.transform(x)

        return x, y

# CNN Models

## Network

In [ ]:
from torchvision.models.video import r3d_18

class SimpleCNN3D(nn.Module):
    def __init__(self, n_classes):
        super(SimpleCNN3D, self).__init__()
        self.conv1 = nn.Conv3d(1, 16, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm3d(16)
        self.pool1 = nn.MaxPool3d(2)

        self.conv2 = nn.Conv3d(16, 32, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm3d(32)
        self.pool2 = nn.MaxPool3d(2)

        self.conv3 = nn.Conv3d(32, 64, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm3d(64)
        self.pool3 = nn.AdaptiveAvgPool3d(1)

        self.dropout = nn.Dropout(p=0.4)
        self.fc = nn.Linear(64, n_classes)

    def forward(self, x):
        x = self.pool1(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool2(torch.relu(self.bn2(self.conv2(x))))
        x = self.pool3(torch.relu(self.bn3(self.conv3(x))))
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        return self.fc(x)


class ResNet3D(nn.Module):
    def __init__(self, n_classes):
        super(ResNet3D, self).__init__()
        self.model = r3d_18(weights=None)
        self.model.stem[0] = nn.Conv3d(1, 64, kernel_size=(3,7,7), stride=(1,2,2), padding=(1,3,3), bias=False)
        self.model.fc = nn.Linear(self.model.fc.in_features, n_classes)

    def forward(self, x):
        return self.model(x)

## Training, Validation and Testing loops

Training function

In [ ]:
def train(train_loader, model, criterion, optimizer, device):
    # Enable training mode 
    model.train()
    running_loss = 0.0

    for x_batch, y_batch in train_loader:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)

        # Reset gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(x_batch)

        # Compute loss
        loss = criterion(outputs, y_batch)

        # Backpropagation
        loss.backward()

        # Update weights
        optimizer.step()

        running_loss += loss.item() * x_batch.size(0)

    train_loss = running_loss / len(train_loader.dataset)
    return train_loss

Validation function

In [ ]:
def validate(val_loader, model, criterion, device, task):
    # Set model to evaluation mode
    model.eval()
    running_loss = 0.0
    correct = 0

    # Disable gradient computation
    with torch.no_grad():
        for x_val, y_val in val_loader:
            x_val, y_val = x_val.to(device), y_val.to(device)
            outputs = model(x_val)
            loss = criterion(outputs, y_val)
            running_loss += loss.item() * x_val.size(0)
            
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == y_val).sum().item()

    val_loss = running_loss / len(val_loader.dataset)
    val_accuracy = correct / len(val_loader.dataset)
    return val_loss, val_accuracy

Evaluation function (to get predictions)

In [ ]:
def evaluate(model, loader, task, device):
    model.eval()
    true_labels, pred_labels = [], []

    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            outputs = model(x)

            # Convert output to prediction
            if task == 'classification':
                preds = torch.argmax(outputs, dim=1)  
            else:
                preds = outputs.squeeze()  

            # Store true and predicted values
            true_labels.extend(y.cpu().numpy())
            pred_labels.extend(preds.cpu().numpy())

    return np.array(true_labels), np.array(pred_labels)

# Data

Load some files for example

In [ ]:
img3D = np.load('/kaggle/input/fcmaps-processed/002_S_4654.processed.npy')

print(img3D.shape)
print(img3D.dtype)

## Pairwise classification

- ADNI + CBS = 116 
- CBS + PSP = 105 
- ADNI + PSP = 133

In [ ]:
group1 = 'ADNI'
group2 = 'PSP'

df_pair = df_labels[df_labels['Group'].isin([group1, group2])].reset_index(drop=True)

In [ ]:
df_pair.shape

## Splitting

In [ ]:
# -------------------- SPLIT TRAIN/TEST  --------------------

subjects = df_pair['ID'].values
labels = df_pair['Group'].values

# Split subjects into train and test subjects
train_subj, test_subj = train_test_split(
    subjects,
    stratify=labels,
    test_size=0.2,
    random_state=42
)

train_df = df_pair[df_pair['ID'].isin(train_subj)]
df_test = df_pair[df_pair['ID'].isin(test_subj)]

# -------------------- SPLIT TRAIN/VALIDATION --------------------

subjects_train = train_df['ID'].values
labels_train = train_df[label_column].values

train_subj, val_subj = train_test_split(
    subjects_train,
    stratify=labels_train,
    test_size=0.2,
    random_state=42
)

df_train = train_df[train_df['ID'].isin(train_subj)]
df_val = train_df[train_df['ID'].isin(val_subj)]

Check the balance between groups

In [ ]:
print("Train set label distribution:")
print(df_train['Group'].value_counts())
print("Validation set label distribution:")
print(df_val['Group'].value_counts())
print("Test set label distribution:")
print(df_test['Group'].value_counts())

Example of augmentation

In [ ]:
train_dataset = AugmentedFCDataset(path_fcmaps_augmented, train_df, label_column, task)

In [ ]:
print("Shape before augmentation: ", train_df.shape )
print("Shape after augmentation: ", len(train_dataset) )

# Training

In [ ]:
# Create Datasets
train_dataset = AugmentedFCDataset(path_fcmaps_augmented, df_train, label_column, task)
val_dataset = FCDataset(path_fcmaps, df_val, label_column, task)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# MODEL: ResNet3D
model = ResNet3D(n_classes=2).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

# Paramters
train_losses = []
val_losses = []
val_accuracies = []

best_accuracy = -float('inf')
best_epoch = -1
best_model_path = None

# Training 
for epoch in range(epochs):
    train_loss = train(train_loader, model, criterion, optimizer, device)
    val_loss, val_accuracy = validate(val_loader, model, criterion, device, task)

    print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Accuracy: {val_accuracy:.4f}")

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

    # Save best model with epoch number in the filename
    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        best_epoch = epoch + 1  
        best_model_path = f'best_model_epoch{best_epoch}.pt'
        torch.save(model.state_dict(), best_model_path)

In [ ]:
def plot_losses_and_accuracy(train_losses, val_losses, val_accuracies):
    plt.figure(figsize=(8, 5))
    plt.plot(train_losses, label='Train Loss', marker='o')
    plt.plot(val_losses, label='Validation Loss', marker='s')
    plt.plot(val_accuracies, label='Validation Accuracy', marker='^')
    plt.xlabel('Epoch')
    plt.ylabel('Value')
    plt.title('Training Loss, Validation Loss and Validation Accuracy')
    plt.legend()
    plt.grid(True)
    plt.show()

def plot_losses_only(train_losses, val_losses):
    plt.figure(figsize=(8, 5))
    plt.plot(train_losses, label='Train Loss', marker='o')
    plt.plot(val_losses, label='Validation Loss', marker='s')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.grid(True)
    plt.show()

# --- PLOT ---
plot_losses_and_accuracy(train_losses, val_losses, val_accuracies)
plot_losses_only(train_losses, val_losses)

# Evaluation

In [ ]:
print(f"\nBest validation accuracy: {best_accuracy:.4f} at epoch {best_epoch}")
print(f"Best model saved at: {best_model_path}")

In [ ]:
# Dataset and DataLoaders 
test_dataset = FCDataset(path_fcmaps, df_test, label_column, task)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Load the model
model = ResNet3D(n_classes=n_classes).to(device)
model.load_state_dict(torch.load(best_model_path, weights_only=True))
model.eval()

# Test
y_true, y_pred = evaluate(model, test_loader, task, device)

In [ ]:
# Metrics
report = classification_report(
    y_true, y_pred,
    output_dict=True,
    zero_division=0  
)

df_report = pd.DataFrame(report).transpose()

In [ ]:
df_report.round(3)

In [ ]:
# Confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.title("Test Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()